In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [6]:
!pip install -q findspark


In [7]:
import findspark
findspark.init() 

In [8]:
!pip install pyspark

In [9]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local[*]").appName("Task 1").getOrCreate()  # use all cores of your CPU - each core has thread

#**Challenge 1 -  PySpark and SQL Querying**

Write a PySpark code to read all the files in the `by-day` folder, and displays a table that shows the ID of the customer and the the total amount they paid for all their purchases. Make sure to drop any stock that has the word *WATER* in it before calculating the total price.

In [10]:
!unzip by-day.zip

Archive:  by-day.zip
  inflating: by-day/2010-12-01.csv   
  inflating: by-day/2010-12-02.csv   
  inflating: by-day/2010-12-03.csv   
  inflating: by-day/2010-12-05.csv   
  inflating: by-day/2010-12-06.csv   
  inflating: by-day/2010-12-07.csv   
  inflating: by-day/2010-12-08.csv   
  inflating: by-day/2010-12-09.csv   
  inflating: by-day/2010-12-10.csv   
  inflating: by-day/2010-12-12.csv   
  inflating: by-day/2010-12-13.csv   
  inflating: by-day/2010-12-14.csv   
  inflating: by-day/2010-12-15.csv   
  inflating: by-day/2010-12-16.csv   
  inflating: by-day/2010-12-17.csv   
  inflating: by-day/2010-12-19.csv   
  inflating: by-day/2010-12-20.csv   
  inflating: by-day/2010-12-21.csv   
  inflating: by-day/2010-12-22.csv   
  inflating: by-day/2010-12-23.csv   
  inflating: by-day/2011-01-04.csv   
  inflating: by-day/2011-01-05.csv   
  inflating: by-day/2011-01-06.csv   
  inflating: by-day/2011-01-07.csv   
  inflating: by-day/2011-01-09.csv   
  inflating: by-day/2011-01-1

In [11]:
df1 = spark.read.format('csv').option("header", "true").option("inferSchema","true").load("by-day/*.csv")

In [12]:
df1.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    21544|SKULLS  WATER TRA...|      48|2011-12-05 08:38:00|     0.85|   14075.0|United Kingdom|
|   580538|    23126|FELTCRA

In [13]:
from pyspark.sql.functions import col
df1 = df1.select(df1.columns).where(~col("Description").like("% WATER %"))
df1.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|
|   580538|    23126|FELTCRAFT GIRL AM...|       8|2011-12-05 08:38:00|     4.95|   14075.0|United Kingdom|
|   580538|    21833|CAMOUFL

Solve the same question using raw SQL.

In [14]:
from pyspark.sql.functions import desc
df2 = df1.selectExpr("CustomerID", "(UnitPrice * Quantity) as total_cost").groupby(col("CustomerID")).sum("total_cost").sort(desc("sum(total_cost)"))

In [15]:
df2.show()

+----------+------------------+
|CustomerID|   sum(total_cost)|
+----------+------------------+
|      null|1369607.5800000506|
|   14646.0| 276588.9899999999|
|   18102.0|256438.48999999993|
|   17450.0|         179989.37|
|   14911.0|129805.12000000016|
|   12415.0|123556.25000000001|
|   14156.0|109706.65000000004|
|   17511.0| 86752.45999999999|
|   13694.0|           62653.1|
|   16684.0|62095.280000000006|
|   15311.0| 59394.58999999995|
|   14096.0| 56220.29999999983|
|   15061.0| 54228.73999999999|
|   13089.0| 53372.35999999998|
|   17949.0|          52750.84|
|   15769.0|          51731.92|
|   16029.0|          50992.61|
|   14298.0| 49660.00999999999|
|   14088.0|          49077.91|
|   17841.0| 39383.76999999996|
+----------+------------------+
only showing top 20 rows



In [16]:
df1 = spark.read.format('csv').option("header", "true").option("inferSchema","true").load("by-day/*.csv")
df1.createOrReplaceTempView('my_retail')

In [17]:
sqlQuery = spark.sql(""" SELECT CustomerID, sum(Quantity* UnitPrice) AS Total from my_retail WHERE Description NOT LIKE '% WATER %' GROUP BY CustomerID ORDER BY Total DESC """)

In [18]:
sqlQuery.show()

+----------+------------------+
|CustomerID|             Total|
+----------+------------------+
|      null|1369607.5800000506|
|   14646.0| 276588.9899999999|
|   18102.0|256438.48999999993|
|   17450.0|         179989.37|
|   14911.0|129805.12000000016|
|   12415.0|123556.25000000001|
|   14156.0|109706.65000000004|
|   17511.0| 86752.45999999999|
|   13694.0|           62653.1|
|   16684.0|62095.280000000006|
|   15311.0| 59394.58999999995|
|   14096.0| 56220.29999999983|
|   15061.0| 54228.73999999999|
|   13089.0| 53372.35999999998|
|   17949.0|          52750.84|
|   15769.0|          51731.92|
|   16029.0|          50992.61|
|   14298.0| 49660.00999999999|
|   14088.0|          49077.91|
|   17841.0| 39383.76999999996|
+----------+------------------+
only showing top 20 rows



#**Challenge 2-  Pipeline**

Applying the following tasks to create your ML pipeline :

1. First, read all the files in the `by-day` folder.
2. Next, create a pipeline using the `Pipeline()` function, in which the indexer is followed by the assembler.
3. Afterwards, fit and transform the data at hand using the pipeline we created which includes: StringIndexer and VectorAssembler. 


In [25]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
sindexer = StringIndexer().setInputCol("day_of_week").setOutputCol("day_of_week_indexed")
va = VectorAssembler().setInputCols(["UnitPrice","Quantity", "day_of_week_indexed"]).setOutputCol("Unit-Quantity-day_of_week")

In [26]:
from pyspark.sql.functions import date_format,col
my_df = df1.withColumn("day_of_week",date_format(col("InvoiceDate"),"EEEE"))

In [27]:
my_df.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|     Monday|
|   580538|    21914|BLUE HARMONICA IN...|      24|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|
|   580538|    22467|   GUMBALL COAT RACK|       6|2011-12-05 08:38:00|     2.55|   14075.0|United Kingdom|     Monday|
|   580538|    21544|SKULLS  WATER TRA..

In [28]:
from pyspark.ml import Pipeline
my_pl = Pipeline().setStages([sindexer,va])

In [29]:
fittedpl = my_pl.fit(my_df)
transpl = fittedpl.transform(my_df)

In [30]:
transpl.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-------------------+-------------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|day_of_week|day_of_week_indexed|Unit-Quantity-day_of_week|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------+-------------------+-------------------------+
|   580538|    23084|  RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|     Monday|                2.0|          [1.79,48.0,2.0]|
|   580538|    23077| DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|     Monday|                2.0|          [1.25,20.0,2.0]|
|   580538|    22906|12 MESSAGE CARDS ...|      24|2011-12-05 08:38:00|     1.65|   14075.0|United Kingdom|     Monday|                2.0|          [1.65,24.0,2.0]|
|   